In [34]:
!pip install requests beautifulsoup4 python-dotenv huggingface_hub


In [ ]:
from huggingface_hub import InferenceClient
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import requests
import os

In [ ]:
os.environ["HF_TOKEN"] = 'token'


In [ ]:

# conversational-capable model
client = InferenceClient(
    model="mistralai/Mistral-7B-Instruct-v0.3",
    token=os.environ["HF_TOKEN"]
)

# Fetch and clean a webpage
headers = {"User-Agent": "Mozilla/5.0"}
class Website:
    def __init__(self, url):
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        self.title = soup.title.string if soup.title else "No title found"
        for tag in soup(["script", "style", "img", "input"]):
            tag.decompose()
        self.text = soup.get_text(separator="\n", strip=True)

In [ ]:

# Prompt builder
def summarize(url):
    website = Website(url)
    system_prompt = (
        "You are an assistant that analyzes the contents of a website "
        "and provides a short summary, ignoring navigation or repetitive elements. "
        "Respond in markdown."
    )
    user_prompt = f"Website title: {website.title}\nContent:\n{website.text[:4000]}"

In [ ]:

    # Conversational call
    response = client.chat_completion(
        model="mistralai/Mistral-7B-Instruct-v0.3",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        max_tokens=500,
    )

    return response.choices[0].message["content"]


In [50]:


# Display function
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

# Test it
display_summary("https://edwarddonner.com")


 **Summary:**

The website belongs to Edward Donner, a tech enthusiast and CTO of Nebula.io, an AI company that helps recruiters discover and manage talent. He is also the co-founder and former CEO of AI startup untapt, which was acquired in 2021. The website features sections for Connect Four, Outsmart (an LLM competition), About, Posts, and a contact page. Edward shares his interests in coding, DJing, electronic music production, and browsing Hacker News. He invites visitors to connect with him and provides links to his LinkedIn, Twitter, and Facebook profiles, as well as a newsletter subscription option. The website was last updated on September 15, 2025, with a post about AI in Production. Other recent posts include topics related to LLMs, AI engineering, and an AI Executive Briefing.